## Magic Formula and auto send orders to buy stocks in MT5

In [1]:
import pandas as pd
import requests
import telebot
import MetaTrader5 as mt5

##### Webscraping

In [19]:
url = 'http://www.fundamentus.com.br/resultado.php'
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}
r = requests.get(url, headers=header)
tabela = pd.read_html(r.text,  decimal=',', thousands='.')[0]

In [ ]:
def acoes_setor(setor=None):
    """
    Setores:
      1 - Agropecuária
      2 - Água e Saneamento
      3 - Alimentos Processados
      4 - Serv.Méd.Hospit. Análises e Diagnósticos
      5 - Automóveis e Motocicletas
      6 - Bebidas
      7 - Comércio
      8 - Comércio e Distribuição
      9 - Computadores e Equipamentos
      10 - Construção Civil
      11 - Construção e Engenharia
      12 - Diversos
      13 - 
      14 - Energia Elétrica
      15 - Equipamentos
      16 - Exploração de Imóveis
      17 - Gás
      18 - Holdings Diversificadas
      19 - Hoteis e Restaurantes
      20 - Intermediários Financeiros
      21 - Madeira e Papel
      22 - Máquinas e Equipamentos
      23 - Materiais Diversos
      24 - Material de Transporte
      25 - Medicamentos e Outros Produtos
      26 - Mídia
      27 - Mineração
      28 - Outros
      29 - 
      30 - Petróleo, Gás e Biocombustíveis
      31 - Previdência e Seguros
      32 - Produtos de Uso Pessoal e de Limpeza
      33 - Programas e Serviços
      34 - Químicos
      35 - 
      36 - Serviços Diversos
      37 - Serviços Financeiros Diversos
      38 - Siderurgia e Metalurgia
      39 - Tecidos, Vestuário e Calçados
      40 - Telecomunicações
      41 - Transporte
      42 - Utilidades Domésticas
      43 - Viagens e Lazer
    
    Output:
      List
    """

    ## GET: setor
    url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
    header = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
    r = requests.get(url, headers=header)
    df = pd.read_html(r.text,  decimal=',', thousands='.')[0]
    return list(df['Papel'])

##### Data handling

In [20]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  tabela[coluna] = tabela[coluna].str.replace('.', '')
  tabela[coluna] = tabela[coluna].str.replace(',', '.')
  tabela[coluna] = tabela[coluna].str.rstrip('%').astype('float') / 100

##### Filtering DataFrame

In [21]:
tabela = tabela[['Papel', 'Cotação', 'EV/EBIT', 'ROIC', 'Liq.2meses', 'P/L']]
tabela['Empresa'] = tabela['Papel'].str[:4]
tabela

,Papel,Cotação,EV/EBIT,ROIC,Liq.2meses,P/L,Empresa
0,CSTB4,147.69,0.00,0.2240,0.0,0.00,CSTB
1,IVTT3,0.00,0.00,0.0000,0.0,0.00,IVTT
2,PORP4,2.40,0.00,0.0000,0.0,0.00,PORP
3,POPR4,10.17,0.00,0.1525,0.0,0.00,POPR
4,MNSA4,0.47,0.00,-0.1350,0.0,0.00,MNSA
...,...,...,...,...,...,...,...
981,PRBC4,14.54,0.00,0.0000,0.0,502.29,PRBC
982,UBBR4,7.49,0.00,0.0000,0.0,610.27,UBBR
983,UBBR11,14.75,0.00,0.0000,0.0,1201.81,UBBR
984,UBBR3,18.00,0.00,0.0000,0.0,1466.61,UBBR


##### Delete finances and insurance companies

In [ ]:
interm_finan = acoes_setor(20)
prev_seg = acoes_setor(31)

In [ ]:
empresas_fora = interm_finan + prev_seg

In [ ]:
mascara = tabela['Papel'].isin(empresas_fora)

In [ ]:
tabela = tabela[~mascara]

##### Finishing data handling

In [23]:
liquidez = 1000000

In [22]:
tabela = tabela.drop_duplicates(subset='Empresa')
tabela = tabela.set_index('Papel')
tabela = tabela[tabela['Liq.2meses'] > liquidez]
tabela = tabela[tabela['P/L'] > 0]
tabela = tabela[tabela['EV/EBIT'] > 0]
tabela = tabela[tabela['ROIC'] > 0]
tabela = tabela.drop(columns = ['Empresa', 'P/L', 'Liq.2meses'])
tabela

,Cotação,EV/EBIT,ROIC
Papel,,,
PETR4,33.87,2.52,0.2895
CMIG4,12.70,4.57,0.1635
JALL3,9.09,28.62,0.0348
ALSO3,24.12,14.18,0.0324
GGBR3,24.02,3.96,0.1939
...,...,...,...
CEAB3,5.52,7.11,0.0528
CRFB3,10.66,8.44,0.0666
YDUQ3,20.90,14.32,0.0899


##### Create Magic Formula Rank

In [ ]:
qtd_ativos = 15

In [24]:
tabela['RANKING_EV/EBIT'] = tabela['EV/EBIT'].rank(ascending = True)
tabela['RANKING_ROIC'] = tabela['ROIC'].rank(ascending = False)
tabela['RANKING_TOTAL'] = tabela['RANKING_EV/EBIT'] + tabela['RANKING_ROIC']
tabela = tabela.sort_values('RANKING_TOTAL')
tabela = tabela.head(qtd_ativos)
tabela

,Cotação,EV/EBIT,ROIC,RANKING_EV/EBIT,RANKING_ROIC,RANKING_TOTAL
Papel,,,,,,
PSSA3,27.03,0.25,0.6076,1.0,1.0,2.0
PETR4,33.87,2.52,0.2895,3.0,6.0,9.0
WIZC3,6.20,2.31,0.2418,2.0,11.0,13.0
KEPL3,12.08,5.39,0.4361,15.0,2.0,17.0
CMIN3,4.43,4.99,0.2849,11.0,7.0,18.0
FESA4,48.65,4.71,0.2557,9.0,10.0,19.0
VLID3,18.38,4.42,0.2064,5.0,15.0,20.0
PLPL3,9.80,5.47,0.3059,16.0,4.0,20.0
GGBR3,24.02,3.96,0.1939,4.0,21.0,25.0


##### Send mensage to Telegram with a stock list

In [ ]:
ranking = tabela.index
ranking = '\n'.join(f'{i+1}. {acao}' for i, acao in enumerate(ranking))
mensagem = f'RANKING DA MAGIC FORMULA:\n{ranking}'
print(mensagem)
bot = telebot.TeleBot("YOUR BOT HERE")
group = "YOUR GROUP HERE"
bot.send_message(group, mensagem)

##### Start MetaTrader 5

Documentação: https://www.mql5.com/en/docs/python_metatrader5

In [10]:
mt5.initialize()

True

##### Create a list of stocks to send order

In [25]:
tickers = tabela.index
tickers

Index(['PSSA3', 'PETR4', 'WIZC3', 'KEPL3', 'CMIN3', 'FESA4', 'VLID3', 'PLPL3',
       'GGBR3', 'PRNR3', 'AURA33', 'CMIG4', 'BEEF3', 'RECV3', 'CSUD3'],
      dtype='object', name='Papel')

##### Send order to buy

In [ ]:
for ticker in tickers:

    print(ticker)

    info_acoes = mt5.symbol_info(ticker)
    mt5.symbol_select(ticker)
    tick_min = mt5.symbol_info(ticker).point
    preco = mt5.symbol_info_tick(ticker).ask
    quantidade = 100.0
    ordem_compra = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": quantidade,
        "type": mt5.ORDER_TYPE_BUY,
        "price": preco,
        "magic": 1,
        "comment": "Fórmula Mágica",
        "type_time": mt5.ORDER_TIME_DAY,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result_compra = mt5.order_send(ordem_compra)
    print(result_compra)